Extracción con el PDF sin texto

In [16]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


Convertir PDF a JPG de alta calidad 

In [ ]:
import fitz
import os

def convert_pdf_to_jpg(pdf_path, output_folder, dpi=600):
    
    doc = fitz.open(pdf_path)
    os.makedirs(output_folder, exist_ok=True)

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        
        zoom = dpi / 72
        mat = fitz.Matrix(zoom, zoom)

        # Render
        pix = page.get_pixmap(matrix=mat)

        output_image_path = os.path.join(output_folder, f"page_{page_num+1}.jpg")
        pix.save(output_image_path)

    return len(doc)


pdf_path = 'instruction_handbook - copia.pdf'  
output_folder = 'imagenes_jpg'  
#funcion que lo convierte a JPG
num_pages = convert_pdf_to_jpg(pdf_path, output_folder)
print(f"Se han convertido {num_pages} páginas a imágenes JPG.")


Se han convertido 54 páginas a imágenes JPG.


Reconvertir las imagenes a PDF

In [ ]:
from PIL import Image
import os
import re

def sort_key_func(file):
    #ordenar
    numbers = re.findall(r'\d+', file)
    return [int(num) for num in numbers]

def convert_images_to_pdf(image_folder, output_pdf_path):
   
    images = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith((".jpg", ".jpeg", ".png"))]
    images.sort(key=sort_key_func)  # Ordena segun el nobre

    image_list = []

    for img_path in images:
        image = Image.open(img_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        image_list.append(image)

    # Crea un PDF
    if image_list:
        image_list[0].save(output_pdf_path, save_all=True, append_images=image_list[1:], quality=100)

image_folder = 'imagenes_jpg'
output_pdf_path = 'instruction_handbook_jpg3.pdf'

convert_images_to_pdf(image_folder, output_pdf_path)
print(f"Las imágenes de '{image_folder}' han sido convertidas a un PDF en '{output_pdf_path}'.")


Las imágenes de 'imagenes_jpg' han sido convertidas a un PDF en 'instruction_handbook_jpg3.pdf'.


Extraccion de imagenes de alta calidad del pdf

In [18]:
import cv2
import numpy as np
import fitz 
import os

def extract_embedded_images(pdf_path, output_folder):
    doc = fitz.open(pdf_path)
    os.makedirs(output_folder, exist_ok=True)
    extracted_image_count = 0

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap()
        output_image_path = os.path.join(output_folder, f"page_{page_num+1}.png")
        pix.save(output_image_path)

        # Lee la página
        page_image = cv2.imread(output_image_path)
        gray = cv2.cvtColor(page_image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

        # Encontrar contornos
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        image_index = 0

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w * h > 500:  # umbral
                embedded_image = page_image[y:y+h, x:x+w]
                embedded_image_path = os.path.join(output_folder, f"pag_{page_num+1}_img_{image_index+1}.png")
                cv2.imwrite(embedded_image_path, embedded_image)
                image_index += 1
                extracted_image_count += 1

    return extracted_image_count


pdf_path = 'instruction_handbook_jpg3.pdf'
output_folder = 'imagenes3' 

# Extraer imágenes
num_images = extract_embedded_images(pdf_path, output_folder)
print(f"Se han extraído {num_images} imágenes incrustadas.")


Se han extraído 2268 imágenes incrustadas.
